In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as sfs


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
def sfsFeature(indep_x,dep_y):
    SFSlist=[]
    SFSlist1=[]
    SFSlist2=[]
     


    log_model = LogisticRegression(solver='lbfgs')
    svc_model=SVC(kernel='linear',max_iter=100,random_state=0)
    DT=DecisionTreeClassifier(criterion='gini',max_features='sqrt',splitter='best',random_state=0)
    RF=RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=0)
    SFSmodellist=[log_model,svc_model,DT,RF]
    

    for model in SFSmodellist:
        log_sfs = sfs(model, k_features=6,forward=False, floating=False, verbose=0, scoring='accuracy', cv=5, n_jobs=1)
        log_fit = log_sfs.fit(indep_x, dep_y)
        log_sfs_feature = log_fit.transform(indep_x)
        SFSlist.append(log_sfs_feature)

        # Get the indices and names of selected features
        op = log_sfs.k_feature_idx_
        op1 = log_sfs.k_feature_names_
        SFSlist1.append(op)
        SFSlist2.append(op1)

    return SFSlist, SFSlist1, SFSlist2
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        
        #Feature Scaling
        #from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        
        return X_train, X_test, y_train, y_test,sc
        
    
def cm_prediction(classifier,X_test):
        y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(y_test, y_pred)
        
        from sklearn.metrics import accuracy_score 
        from sklearn.metrics import classification_report 
        
        Accuracy=accuracy_score(y_test, y_pred )
        
        report=classification_report(y_test, y_pred)
        return  classifier,Accuracy,report,X_test,y_test,cm


In [4]:
def logistic(X_train,y_train,X_test):
    
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      
    
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'linear', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

def Navie(X_train,y_train,X_test):   
    
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm         
    
    
def knn(X_train,y_train,X_test):
           
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def Decision(X_train,y_train,X_test):
        
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      


def random(X_train,y_train,X_test):
        
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    

def sfs_classification(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf):
   
    sfsdataframe = pd.DataFrame(index=['Logistic','SVC','DT','RF'],
                                 columns=['Logistic', 'SVMl', 'SVMnl', 'KNN', 'Navie', 'Decision', 'Random'])
    for number,idex in enumerate(sfsdataframe.index):
        
        sfsdataframe['Logistic'][idex]=acclog[number]       
        sfsdataframe['SVMl'][idex]=accsvml[number]
        sfsdataframe['SVMnl'][idex]=accsvmnl[number]
        sfsdataframe['KNN'][idex]=accknn[number]
        sfsdataframe['Navie'][idex]=accnav[number]
        sfsdataframe['Decision'][idex]=accdes[number]
        sfsdataframe['Random'][idex]=accrf[number]
    

    return sfsdataframe

In [8]:
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, drop_first=True)

indep_X=df2.drop('classification_yes', axis=1)
dep_Y=df2['classification_yes']

In [11]:
SFSlist=sfsFeature(indep_X,dep_Y)       

acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]

In [12]:
SFSlist, SFSlist1,SFSlist2=SFSlist

In [13]:
SFSlist

[array([[ 76.45994832,   3.        , 148.11267606,   3.07735602,
          12.51815562,   0.        ],
        [ 76.45994832,   2.        , 148.11267606,   0.7       ,
          10.7       ,   0.        ],
        [ 76.45994832,   1.        ,  99.        ,   0.6       ,
          12.        ,   0.        ],
        ...,
        [ 70.        ,   3.        , 110.        ,   6.        ,
           9.1       ,   1.        ],
        [ 90.        ,   0.        , 207.        ,   6.8       ,
           8.5       ,   1.        ],
        [ 80.        ,   0.        , 100.        ,   1.        ,
          16.3       ,   0.        ]]),
 array([[ 3.        ,  3.07735602, 12.51815562,  1.        ,  0.        ,
          0.        ],
        [ 2.        ,  0.7       , 10.7       ,  1.        ,  0.        ,
          0.        ],
        [ 1.        ,  0.6       , 12.        ,  0.        ,  0.        ,
          0.        ],
        ...,
        [ 3.        ,  6.        ,  9.1       ,  1.        ,  0

In [14]:
 SFSlist1

[(1, 2, 4, 6, 9, 21),
 (2, 6, 9, 14, 15, 22),
 (2, 10, 14, 15, 22, 23),
 (6, 9, 13, 15, 16, 17)]

In [15]:
SFSlist2

[('bp', 'al', 'bgr', 'sc', 'hrmo', 'htn_yes'),
 ('al', 'sc', 'hrmo', 'sg_c', 'sg_d', 'dm_yes'),
 ('al', 'pcv', 'sg_c', 'sg_d', 'dm_yes', 'cad_yes'),
 ('sc', 'hrmo', 'sg_b', 'sg_d', 'sg_e', 'rbc_normal')]

In [16]:
for i in SFSlist:   
    
    X_train, X_test, y_train, y_test,sc=split_scalar(i,dep_Y)   
    
        
    classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
    acclog.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=svm_linear(X_train,y_train,X_test)  
    accsvml.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=svm_NL(X_train,y_train,X_test)  
    accsvmnl.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=knn(X_train,y_train,X_test)  
    accknn.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=Navie(X_train,y_train,X_test)  
    accnav.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=Decision(X_train,y_train,X_test)  
    accdes.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
    accrf.append(Accuracy)
result=sfs_classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf)


In [17]:
result
#6

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
Logistic,0.97,0.96,0.97,0.97,0.91,0.98,0.98
SVC,0.98,0.98,0.98,0.98,0.97,0.96,1
DT,0.96,0.96,0.96,0.99,0.96,0.99,0.99
RF,0.96,0.97,0.96,0.98,0.73,0.94,0.97
